In [6]:
import os, sys
from pathlib import Path
import numpy as np
import pandas as pd
sys.path.append(str("/home/alessio/dr_sasa_python/build/lib"))
sys.path.append(str("/home/alessio/dr_sasa_python/bindings/python/utils"))
import dr_sasa_py
from structure_parser import *
from utils import convert_to_dataframes

In [ ]:
test_pdb = "/home/alessio/dr_sasa_python/tests/data/pred.pdb"
calc = dr_sasa_py.GenericSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb), include_matrix=True, print_output=True)
dfs = convert_to_dataframes(result)
overlaps = dfs["overlaps"]
atoms = dfs["atoms"]
residues = dfs["residues"]
overlaps[overlaps["source_residue"].str.startswith("A_")].overlap_area.sum() #A <--- B buried surface (A^2):	791.494
result["intra_bsa_matrix"]["residue_matrix"][:len(residues[residues.chain =="A"].index)].sum() # same as #A <--- B buried surface (A^2):	791.494
result["inter_bsa_matrix"]["A_vs_B"]["matrix"].sum()
##
overlaps[overlaps["source_residue"].str.startswith("B_")].overlap_area.sum() #A ---> B buried surface (A^2):	741.817
result["intra_bsa_matrix"]["residue_matrix"][len(residues[residues.chain =="A"].index):].sum() # same as A ---> B buried surface (A^2):	741.817
result["inter_bsa_matrix"]["B_vs_A"]["matrix"].sum()
##
overlaps.overlap_area.sum()/2 #Interface A/B (A^2):	766.656
result["intra_bsa_matrix"]["atom_matrix"].sum()/2
##
atoms[atoms.chain == "A"].sasa.sum() #Object A complexed surface (A^2):	1198.94 calculation based on atom contact surfaces
residues[residues.chain == "A"].dsasa.sum() # close values (calculated based on standards NACESS sasa values of per residues)
atoms[atoms.chain == "B"].sasa.sum() #Object B complexed surface (A^2):	2165.03
## 
atoms.sasa.sum() #Selected complex surface (A^2):	3363.97
#
overlaps[overlaps["source_residue"].str.startswith("A_")].overlap_area.sum() + atoms[atoms.chain == "A"].sasa.sum() #  Object A uncomplexed surface (A^2):	1990.44
residues[residues.chain == "A"].dsasa.sum()


In [40]:
test_pdb = "/home/alessio/dr_sasa_python/tests/data/8wwl.pdb"
calc = dr_sasa_py.GenericSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb),include_matrix=True, print_output=False)

In [ ]:
np.asarray([result["atom_data"][str(i)]["sasa"] for i in list(result["atom_data"].keys())])

[36.734901428222656,
 13.61657428741455,
 1.8416906595230103,
 4.837236404418945,
 12.6360445022583,
 30.394296646118164,
 64.39949035644531,
 0.6294841766357422,
 0.03410538285970688,
 0.03410538285970688,
 11.534948348999023,
 0.2557903826236725,
 7.2218146324157715,
 27.309886932373047,
 38.57318878173828,
 0.0,
 3.22295880317688,
 0.8441082239151001,
 0.0,
 0.0,
 0.0,
 1.7310816049575806,
 5.0475969314575195,
 0.0,
 4.647924900054932,
 1.61147940158844,
 0.0,
 0.0,
 1.18516206741333,
 2.030203104019165,
 7.085392951965332,
 0.0,
 0.0852634608745575,
 4.817652225494385,
 0.0,
 0.0,
 1.0657932758331299,
 0.0,
 0.2088954895734787,
 2.54085111618042,
 7.324131011962891,
 19.818960189819336,
 8.062060356140137,
 0.0,
 1.1425303220748901,
 1.8502171039581299,
 5.855601787567139,
 15.986898422241211,
 4.578647613525391,
 14.8187894821167,
 16.532583236694336,
 55.896697998046875,
 0.5920149087905884,
 1.8928487300872803,
 0.0,
 4.458613395690918,
 7.81865930557251,
 0.0,
 0.05968442186713

In [39]:
for i in result["atom_data"].keys():  # keys are strings, no need to convert
    try:
        result["atom_data"][str(i)]["sasa"]
    except:
        print(i)


In [35]:
result["atom_data"]

{'1': {'name': 'N',
  'resname': 'THR',
  'chain': 'A',
  'resid': 4,
  'struct_type': 'A',
  'coords': (145.60299682617188, 113.54900360107422, 144.49400329589844),
  'sphere_area': 113.09732818603516,
  'sasa': 36.734901428222656,
  'polar': 1,
  'charge': ''},
 '2': {'name': 'CA',
  'resname': 'THR',
  'chain': 'A',
  'resid': 4,
  'struct_type': 'A',
  'coords': (146.23399353027344, 112.93699645996094, 145.6580047607422),
  'sphere_area': 128.6796112060547,
  'sasa': 13.61657428741455,
  'polar': 0,
  'charge': ''},
 '3': {'name': 'C',
  'resname': 'THR',
  'chain': 'A',
  'resid': 4,
  'struct_type': 'A',
  'coords': (145.58799743652344, 113.41999816894531, 146.95199584960938),
  'sphere_area': 128.6796112060547,
  'sasa': 3.4616963863372803,
  'polar': 0,
  'charge': ''},
 '4': {'name': 'O',
  'resname': 'THR',
  'chain': 'A',
  'resid': 4,
  'struct_type': 'A',
  'coords': (146.25, 113.52799987792969, 147.98300170898438),
  'sphere_area': 98.52033996582031,
  'sasa': 23.67700004

In [15]:
result["atom_data"].keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157'

In [ ]:
dfs = convert_to_dataframes(result)
overlaps = dfs["overlaps"]
atoms = dfs["atoms"]
residues = dfs["residues"]

In [ ]:
test_pdb = "/home/alessio/dr_sasa_python/PRODIGYdataset/1FFW.pdb"
calc = dr_sasa_py.SimpleSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb))
result

In [ ]:
test_pdb = "/home/alessio/dr_sasa_python/PRODIGYdataset/1E6E.pdb"
calc = dr_sasa_py.GenericSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb), include_matrix=True, print_output=True)
dfs = convert_to_dataframes(result)
overlaps = dfs["overlaps"]
atoms = dfs["atoms"]
residues = dfs["residues"]
overlaps[overlaps["source_residue"].str.startswith("A_")].overlap_area.sum() #A <--- B buried surface (A^2):	791.494
result["intra_bsa_matrix"]["residue_matrix"][:len(residues[residues.chain =="A"].index)].sum() # same as #A <--- B buried surface (A^2):	791.494
result["inter_bsa_matrix"]["A_vs_B"]["matrix"].sum()
##
overlaps[overlaps["source_residue"].str.startswith("B_")].overlap_area.sum() #A ---> B buried surface (A^2):	741.817
result["intra_bsa_matrix"]["residue_matrix"][len(residues[residues.chain =="A"].index):].sum() # same as A ---> B buried surface (A^2):	741.817
result["inter_bsa_matrix"]["B_vs_A"]["matrix"].sum()
##
overlaps.overlap_area.sum()/2 #Interface A/B (A^2):	766.656
result["intra_bsa_matrix"]["atom_matrix"].sum()/2
##
atoms[atoms.chain == "A"].sasa.sum() #Object A complexed surface (A^2):	1198.94 calculation based on atom contact surfaces
residues[residues.chain == "A"].dsasa.sum() # close values (calculated based on standards NACESS sasa values of per residues)
atoms[atoms.chain == "B"].sasa.sum() #Object B complexed surface (A^2):	2165.03
## 
atoms.sasa.sum() #Selected complex surface (A^2):	3363.97
#
overlaps[overlaps["source_residue"].str.startswith("A_")].overlap_area.sum() + atoms[atoms.chain == "A"].sasa.sum() #  Object A uncomplexed surface (A^2):	1990.44
residues[residues.chain == "A"].dsasa.sum()

In [ ]:
residues[residues.chain == "A"].dsasa.sum() - residues[residues.chain == "A"].total_sasa.sum()
residues[residues.chain == "B"].dsasa.sum() - residues[residues.chain == "B"].total_sasa.sum()

In [ ]:
residues.dsasa.sum() - 

In [ ]:
residues[residues.chain == "B"].dsasa.sum()

In [ ]:
residues[residues.chain == "B"].total_sasa.sum()

In [ ]:
result["intra_bsa_matrix"]["residue_matrix"][:21].sum()
result["inter_bsa_matrix"]["B_vs_A"]["matrix"].sum()

In [ ]:
residues[residues.chain == "A"].dsasa.sum()

In [ ]:
residues = dfs["residues"]
residues[residues.chain == "A"].standard_sasa.sum()

Object A complexed surface (A^2):	1201.52
Object B complexed surface (A^2):	2161.36
Object A uncomplexed surface (A^2):	1998.48
Object B uncomplexed surface (A^2):	2905.35
A <--- B buried surface (A^2):	796.957
A ---> B buried surface (A^2):	743.99
Interface A/B (A^2):	770.474

Selected complex surface (A^2):	3363.97
Object A complexed surface (A^2):	1198.94
Object B complexed surface (A^2):	2165.03
Object A uncomplexed surface (A^2):	1990.44
Object B uncomplexed surface (A^2):	2906.85
A <--- B buried surface (A^2):	791.494
A ---> B buried surface (A^2):	741.817
Interface A/B (A^2):	766.656

In [ ]:
test_pdb = "/home/alessio/dr_sasa_python/tests/data/3i40.pdb"
calc = dr_sasa_py.DecoupledSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb), chains=[['B'], ['C']], print_output=True)

In [ ]:
pd.read_csv("/home/alessio/dr_sasa_python/tests/test_results/3i40.atmasa", sep="\t")
pd.read_csv("/home/alessio/dr_sasa_python/my_analysis", sep="\t")